In [1]:
import os
import sys
sys.path.append('common')
import util, audio_preprocessing

import shutil
import numpy as np
import keras
import keras.backend as K
from keras.callbacks import ModelCheckpoint, Callback, TensorBoard
from matplotlib import pyplot as plt
%matplotlib inline
from cyclic_lr import CyclicLR

from dataset import SingleDataset, MultiDataset
from model import model_mlt_cnn_alexnet

TRY = 'MIX8'
BASE = 'MIX5'
LOG_DIR = './%slog' % TRY
SCD_PREFIX = 'scd_'
USD_PREFIX = 'usd_'

batch_size = 32
epochs = 20

scd = SingleDataset(SCD_PREFIX,
                   util.load_txt_list(os.path.join('.', SCD_PREFIX+'classes.txt')),
                   batch_size)
usd = SingleDataset(USD_PREFIX,
                   ['air_conditioner',  'car_horn', 'children_playing', 'dog_bark', 'drilling',
                    'engine_idling', 'gun_shot','jackhammer', 'siren', 'street_music'],
                   batch_size)

Using TensorFlow backend.


In [2]:
# Clean up log
shutil.rmtree(LOG_DIR, ignore_errors=True)

In [3]:
# Fine tune for scd
multi = MultiDataset([scd, usd], [1.0, 0.0])

model = model_mlt_cnn_alexnet(multi.input_shape(), multi.ys_classes(), freeze_mode=1)
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])
model.load_weights('model%s.h5' % BASE) ### Starting point

callbacks = [
    CyclicLR(base_lr=0.00007, max_lr=0.0007, step_size=multi.train_steps_per_epoch, mode='triangular'),
    ModelCheckpoint('model%s.h5' % TRY,
                monitor='val_output_1_acc',
                verbose=1,
                save_best_only=True,
                save_weights_only=True),
     keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=True, write_images=True)
]
model.fit_generator(multi.train_generator,
                    steps_per_epoch=multi.train_steps_per_epoch,
                    epochs=epochs,
                    validation_data=multi.valid_generator, 
                    validation_steps=multi.valid_steps_per_epoch,
                    callbacks=callbacks)

multi.evaluate_by_datasets(model)

# Fine tune for usd
multi = MultiDataset([scd, usd], [0.0, 1.0])

model = model_mlt_cnn_alexnet(multi.input_shape(), multi.ys_classes(), freeze_mode=2)
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])
model.load_weights('model%s.h5' % TRY)

callbacks = [
    CyclicLR(base_lr=0.00007, max_lr=0.0007, step_size=multi.train_steps_per_epoch, mode='triangular'),
    ModelCheckpoint('model%s.h5' % TRY,
                monitor='val_output_2_acc',
                verbose=1,
                save_best_only=True,
                save_weights_only=True),
     keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=True, write_images=True)
]
model.fit_generator(multi.train_generator,
                    steps_per_epoch=multi.train_steps_per_epoch,
                    epochs=epochs,
                    validation_data=multi.valid_generator, 
                    validation_steps=multi.valid_steps_per_epoch,
                    callbacks=callbacks)

multi.evaluate_by_datasets(model)

model_mlt_cnn_alexnet: freeze other than output_1.
Epoch 1/20
1595/1596 [============================>.] - ETA: 0s - loss: 1.7366 - output_1_loss: 1.7501 - output_2_loss: -0.0135 - output_1_acc: 0.7343 - output_2_acc: 0.1398
Epoch 00001: val_output_1_acc improved from -inf to 0.92822, saving model to modelMIX8.h5
1596/1596 [==============================] - 72s 45ms/step - loss: 1.7372 - output_1_loss: 1.7503 - output_2_loss: -0.0131 - output_1_acc: 0.7344 - output_2_acc: 0.1398 - val_loss: 0.2587 - val_output_1_loss: 0.2786 - val_output_2_loss: -0.0199 - val_output_1_acc: 0.9282 - val_output_2_acc: 0.0977
Epoch 2/20
1595/1596 [============================>.] - ETA: 0s - loss: 1.7272 - output_1_loss: 1.7486 - output_2_loss: -0.0214 - output_1_acc: 0.7348 - output_2_acc: 0.1419
Epoch 00002: val_output_1_acc did not improve
1596/1596 [==============================] - 71s 45ms/step - loss: 1.7274 - output_1_loss: 1.7487 - output_2_loss: -0.0214 - output_1_acc: 0.7347 - output_2_acc: 0.14

Epoch 17/20
1595/1596 [============================>.] - ETA: 0s - loss: 1.7239 - output_1_loss: 1.7376 - output_2_loss: -0.0138 - output_1_acc: 0.7368 - output_2_acc: 0.1419
Epoch 00017: val_output_1_acc did not improve
1596/1596 [==============================] - 72s 45ms/step - loss: 1.7237 - output_1_loss: 1.7375 - output_2_loss: -0.0138 - output_1_acc: 0.7367 - output_2_acc: 0.1420 - val_loss: 0.2578 - val_output_1_loss: 0.2730 - val_output_2_loss: -0.0152 - val_output_1_acc: 0.9313 - val_output_2_acc: 0.0872
Epoch 18/20
1595/1596 [============================>.] - ETA: 0s - loss: 1.7439 - output_1_loss: 1.7477 - output_2_loss: -0.0038 - output_1_acc: 0.7359 - output_2_acc: 0.1408
Epoch 00018: val_output_1_acc did not improve
1596/1596 [==============================] - 71s 45ms/step - loss: 1.7437 - output_1_loss: 1.7476 - output_2_loss: -0.0040 - output_1_acc: 0.7359 - output_2_acc: 0.1407 - val_loss: 0.2593 - val_output_1_loss: 0.2766 - val_output_2_loss: -0.0173 - val_output_1

Epoch 12/20
1595/1596 [============================>.] - ETA: 0s - loss: 1.4116 - output_1_loss: -0.0020 - output_2_loss: 1.4136 - output_1_acc: 0.0356 - output_2_acc: 0.6370
Epoch 00012: val_output_2_acc did not improve
1596/1596 [==============================] - 72s 45ms/step - loss: 1.4121 - output_1_loss: -0.0017 - output_2_loss: 1.4138 - output_1_acc: 0.0356 - output_2_acc: 0.6370 - val_loss: 1.4328 - val_output_1_loss: 0.0042 - val_output_2_loss: 1.4286 - val_output_1_acc: 0.0315 - val_output_2_acc: 0.5597
Epoch 13/20
1595/1596 [============================>.] - ETA: 0s - loss: 1.3720 - output_1_loss: -0.0306 - output_2_loss: 1.4026 - output_1_acc: 0.0348 - output_2_acc: 0.6329
Epoch 00013: val_output_2_acc improved from 0.57242 to 0.57756, saving model to modelMIX8.h5
1596/1596 [==============================] - 72s 45ms/step - loss: 1.3718 - output_1_loss: -0.0308 - output_2_loss: 1.4025 - output_1_acc: 0.0349 - output_2_acc: 0.6328 - val_loss: 1.3338 - val_output_1_loss: 0.00

[0.9315975286849073, 0.5707070707070707]

## Evaluate per dataset

In [6]:
model.load_weights('model%s.h5' % TRY)
multi.evaluate_by_datasets(model)

Evaluating scd_dataset
6798/6798 [==============================] - 11s 2ms/step
 = 0.9315975286849073
Evaluating usd_dataset
990/990 [==============================] - 1s 1ms/step
 = 0.591919191919192
Accuracy with scd_dataset = 0.931598
Accuracy with usd_dataset = 0.591919


[0.9315975286849073, 0.591919191919192]

## Evaluate by mixed dataset

In [8]:
model.load_weights('model%s.h5' % TRY)
multi = MultiDataset([scd, usd], [0.6, 0.4], mix_randomness=0.0)
results = model.evaluate_generator(multi.valid_generator, steps=multi.valid_steps_per_epoch)
print(model.metrics_names, results)

['loss', 'output_1_loss', 'output_2_loss', 'output_1_acc', 'output_2_acc'] [0.8764950239970615, 0.2698882808780865, 0.6066067431976614, 0.8957370313302516, 0.48266563944530044]
